# Pump it Up Data Mining the Water Table
## Import and reading of data

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pump_training_values = pd.read_csv('Pump_it_up_data/training_values.csv')
pump_training_labels = pd.read_csv('Pump_it_up_data/training_labels.csv')
pump_test_values = pd.read_csv('Pump_it_up_data/test_values.csv')

## Getting an overview of the data

Atom is also used to view the cvs files

In [ ]:
print(pump_training_values.describe())

In [ ]:
print(pump_training_values.info())

In [ ]:
pump_training_values['scheme_name']

## Preprocessing

#### Filling the NaN values with the mean value from each collum

In [ ]:
print(pump_training_values.isnull().sum())

pump_training_values.fillna(pump_training_values.mean(), inplace=True)
pump_test_values.fillna(pump_test_values.mean(), inplace=True)

print(pump_training_values.isnull().sum())

In [ ]:
pump_training_values['permit']

In [ ]:
for col in pump_training_values:
    if pump_training_values[col].dtypes == 'object':
        pump_training_values = pump_training_values.fillna(pump_training_values[col].value_counts().index[0])
        
for col in pump_test_values:
    if pump_test_values[col].dtypes == 'object':
        pump_test_values = pump_test_values.fillna(pump_test_values[col].value_counts().index[0])

In [ ]:
print(pump_training_values.isnull().sum())

print(pump_test_values.isnull().sum())

In [ ]:
pump_training_values

## Feature Selection
<ol>
    <li>wpt_name</li>
    <li>num_private</li>
    <li>scheme_name - 28166 values are missing</li>

In [ ]:
pump_training_values = pump_training_values.drop(['id','wpt_name', 'num_private', 'scheme_name','date_recorded'], axis = 1)
pump_test_values = pump_test_values.drop(['id','wpt_name', 'num_private', 'scheme_name','date_recorded'], axis = 1)

### Converting non-numeric values to numeric

Label Encoder from scikit-learn - Out commented

Get_Dummies from pandas

In [ ]:
le = LabelEncoder()

for col in pump_training_values:
    if pump_training_values[col].dtypes == 'object':
        pump_training_values[col] = le.fit_transform(pump_training_values[col])

for col in pump_test_values:
    if pump_test_values[col].dtypes == 'object':
        pump_test_values[col] = le.fit_transform(pump_test_values[col])
        
        
pump_training_labels_1d = pump_training_labels.drop(['id'], axis = 1)

for col in pump_training_labels_1d:
    if pump_training_labels_1d[col].dtypes == 'object':
        pump_training_labels_1d[col] = le.fit_transform(pump_training_labels_1d[col])

In [ ]:
#pd.get_dummies(pump_training_values)
#pd.get_dummies(pump_test_values)

In [ ]:
pump_training_values.info()

In [ ]:
pump_training_values.describe()

## Data Scaling

In [ ]:
scaler = MinMaxScaler(feature_range = (-1,1))
rescaled_pump_training_values = scaler.fit_transform(pump_training_values)
rescaled_pump_test_values = scaler.fit_transform(pump_test_values)

## Model fitting, writing to cvs with proper submission layout

In [ ]:
pump_training_labels_1d = pump_training_labels_1d.to_numpy()

In [ ]:
#logreg = LogisticRegression(max_iter = 1000)
rfc = RandomForestClassifier()
rfc.fit(rescaled_pump_training_values, pump_training_labels_1d.ravel())
#logreg.fit(rescaled_pump_training_values, pump_training_labels_1d.ravel())

In [ ]:
#pump_pred_labels = logreg.predict(rescaled_pump_test_values)
pump_pred_labels = rfc.predict(rescaled_pump_test_values)

In [ ]:
pump_pred_labels

In [ ]:
submission_format = pd.read_csv('Pump_it_up_data/SubmissionFormat.csv')

data = {'status_group':pump_pred_labels}
index = submission_format['id']

index

In [ ]:
submission = pd.DataFrame(data, index)

In [ ]:
submission = submission.replace(0, 'functional')
submission = submission.replace(1, 'functional needs repair')
submission = submission.replace(2, 'non functional')

In [ ]:
submission.to_csv('Pump_it_up_data/pred_labels_RandomForest.csv')